In [3]:
import pandas as pd

trades_df = pd.read_csv('../tmp-data/trade-stats-all-in-one.csv')
trades_df.head(1)

,Unnamed: 0,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Tag,Duration
0,0,-1,59,61,40989.713396,41115.0,-125.286604,-0.003057,2022-04-13 20:09:00,2022-04-13 20:16:00,NaN,0 days 00:07:00


In [6]:
trades_df.sort_values(by='PnL', ascending=True, inplace=True)
trades_df.head(100)


,Unnamed: 0,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Tag,Duration
20164,20164,-4552416366,289245,289288,30008.794660,30233.908000,-1.024810e+12,-0.007502,2023-04-11 17:27:00,2023-04-11 18:12:00,NaN,0 days 00:45:00
18914,18914,-2406000658,271438,271445,26547.533350,26934.117143,-9.301209e+11,-0.014562,2023-03-18 16:33:00,2023-03-18 16:40:00,NaN,0 days 00:07:00
20163,20163,-4077640685,289246,289288,30008.794660,30233.908000,-9.179313e+11,-0.007502,2023-04-11 17:28:00,2023-04-11 18:12:00,NaN,0 days 00:44:00
19535,19535,-3813622632,280309,280316,28013.593400,28248.385714,-8.954093e+11,-0.008381,2023-03-30 16:20:00,2023-03-30 16:30:00,NaN,0 days 00:10:00
18919,18919,-2216344405,271486,271499,26891.630090,27278.317143,-8.570317e+11,-0.014379,2023-03-18 17:38:00,2023-03-18 17:52:00,NaN,0 days 00:14:00
...,...,...,...,...,...,...,...,...,...,...,...,...
19529,19529,-1963050622,280315,280316,28013.593400,28248.385714,-4.609092e+11,-0.008381,2023-03-30 16:29:00,2023-03-30 16:30:00,NaN,0 days 00:01:00
19580,19580,-3784269243,281351,281355,28337.303401,28459.000000,-4.605327e+11,-0.004295,2023-03-31 20:31:00,2023-03-31 20:48:00,NaN,0 days 00:17:00
19488,19488,-3767475425,279775,279785,28267.824251,28389.500000,-4.584104e+11,-0.004304,2023-03-29 22:02:00,2023-03-29 22:31:00,NaN,0 days 00:29:00
20099,20099,-3932860842,288348,288360,28193.985266,28310.500000,-4.582362e+11,-0.004133,2023-04-10 13:53:00,2023-04-10 14:47:00,NaN,0 days 00:54:00


In [7]:
import numpy as np


entry_exit_times = np.array(list(zip(trades_df['EntryTime'].head(10), trades_df['ExitTime'].head(10))))
entry_exit_times

array([['2023-04-11 17:27:00', '2023-04-11 18:12:00'],
       ['2023-03-18 16:33:00', '2023-03-18 16:40:00'],
       ['2023-04-11 17:28:00', '2023-04-11 18:12:00'],
       ['2023-03-30 16:20:00', '2023-03-30 16:30:00'],
       ['2023-03-18 17:38:00', '2023-03-18 17:52:00'],
       ['2023-03-18 16:34:00', '2023-03-18 16:40:00'],
       ['2023-04-11 17:29:00', '2023-04-11 18:12:00'],
       ['2023-03-23 18:56:00', '2023-03-23 19:43:00'],
       ['2023-03-30 16:24:00', '2023-03-30 16:30:00'],
       ['2023-03-26 21:17:00', '2023-03-26 22:12:00']], dtype='<U19')